# Calculating Hotel Occupancy

---

The length of a guest's stay is a key factor in understanding booking patterns and can offer valuable insights into customer behavior, revenue management, and the likelihood of cancellations.

However, the dataset does not directly provide the total stay duration for each reservation. Instead, it includes separate counts for weekday and weekend nights.

I will calculate the total stay duration by summing the number of weekday and weekend nights for each booking. This derived feature will be crucial for further analysis, enabling more accurate modeling of guest behavior, forecasting occupancy rates, and enhancing predictions related to cancellations and revenue.

---

# Import Packages and Read Data

In [7]:
## Data Handling
import datetime as dt
import pandas as pd

## Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

## Load Pre-Reviewed Data

In [8]:
data_path = '../../data/3.1_temporally_updated_data.parquet'
df_data = pd.read_parquet(data_path)
df_data

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,ArrivalDate,DepartureDate,Length of Stay,BookingDate,ArrivalDate_DaysAfterHoliday,ArrivalDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,ArrivalDate_IsHoliday,DepartureDate_IsHoliday,BookingDate_IsHoliday,ArrivalDate_Season,ArrivalDate_DayOfWeek,ArrivalDate_IsWeekend,ArrivalDate_DaysUntilWeekend,ArrivalDate_DaysSinceWeekend,ArrivalDate_Quarter,ArrivalDate_Month,ArrivalDate_WeekOfYear,ArrivalDate_DayOfWeek_Sin,ArrivalDate_DayOfWeek_Cos,ArrivalDate_Month_Sin,ArrivalDate_Month_Cos,ArrivalDate_DayOfYear,ArrivalDate_DayOfYear_Sin,ArrivalDate_DayOfYear_Cos,DaysSinceStart,TimeTrend_Normalized,TimeTrend_Squared,TimeTrend_Cubed,Fourier_Annual_Sin_1,Fourier_Annual_Cos_1,Fourier_SemiAnnual_Sin_1,Fourier_SemiAnnual_Cos_1,Fourier_Quarterly_Sin_1,Fourier_Quarterly_Cos_1,Fourier_Monthly_Sin_1,Fourier_Monthly_Cos_1,Fourier_Weekly_Sin_1,Fourier_Weekly_Cos_1,Interaction_Trend_Annual_Sin,Interaction_Trend_Annual_Cos,Interaction_Trend_Month_Sin,Interaction_Trend_Month_Cos,Interaction_LeadTime_Annual_Sin,Interaction_LeadTime_Annual_Cos,Fourier_Annual_Sin_2,Fourier_Annual_Cos_2,Fourier_Annual_Sin_3,Fourier_Annual_Cos_3,Fourier_Monthly_Sin_2,Fourier_Monthly_Cos_2
0,0,342,2015,July,27,1,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,2015-07-01,2015-07-01,0,2014-07-24,21,45,21,45,44,22,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,342,0.00,1.00,0.00,1.00,0.00,1.00
1,0,257,2015,July,27,1,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6,NULL,0,Transient,80.00,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00
2,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00
3,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00
4,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,108,2017,August,35,31,2,0.00,0,HB,GBR,Online TA,TA/TO,0,0,

# Calculate Daily Occupancies

#### Workflow Walkthrough


**1. Generate Counts for Arrivals and Departures**

```python
arrivals = df_data_h1['ArrivalDate'].value_counts().rename('count')
departures = df_data_h1['DepartureDate'].value_counts().rename('count')
```
- Counts how many reservations start (`arrivals`) and end (`departures`) on each date. The `value_counts()` method tallies occurrences of each date, and `rename('count')` changes the Series name to `'count'`, which aids in clarity for later operations.

**2. Combine Arrival and Departure Counts**

```python
df_counts = pd.concat([arrivals, -departures]).sort_index().reset_index()
df_counts.columns = ['Date', 'Count']
```
- Combines the arrivals and departures into a single DataFrame, `df_counts`, with arrivals contributing positively to the count and departures negatively (indicating the end of a reservation). The data is then sorted by date.

**3. Aggregate Counts on the Same Date**

```python
df_counts = df_counts.groupby('Date').sum()
```
- Since the combination of arrivals and departures could result in multiple entries for the same date, this step aggregates (sums) all counts for each date. This ensures each date is unique, addressing the initial issue of duplicate labels.

**4. Generate a Complete Date Range and Reindex**

```python
date_range = pd.date_range(start=df_counts.index.min(), end=df_counts.index.max())
df_counts = df_counts.reindex(date_range, fill_value=0)
```
- Creates a continuous range of dates covering the entire period from the earliest to the latest date in `df_counts`. It then reindexes `df_counts` to include every date in this range, filling any dates without data with `0`, ensuring there's a record for every single day in the period.

**5. Calculate Cumulative Sum for Active Reservations**

```python
df_counts['Active_Reservations'] = df_counts['Count'].cumsum()
```
- Computes the cumulative sum of the daily net reservation counts (`Count`). This step effectively calculates the total number of active reservations for each date by adding up the arrivals and subtracting the departures as they occur over time.

**Conclusion**

- The final output, `df_counts['Active_Reservations']`, shows the total number of active reservations for each date in the range. This method is efficient and avoids the problem of duplicate labels by ensuring that each date is unique before the reindexing step, leveraging pandas' capabilities for handling time series data.

#### Groupby.Sum vs. Cumsum


The use of both `groupby().sum()` and the `cumsum()` methods serves two different purposes in the process of calculating the total number of active reservations for each date. Here's a clarification of the roles each step plays in the computation:

**1. GroupBy().sum()**

- **Purpose:** This step aggregates the daily net changes in reservations (arrivals and departures) for each unique date. Since arrivals are counted positively and departures negatively, the sum for each date tells us the net reservation change on that day. 
- **What It Solves:** If, for instance, 5 reservations start (arrive) and 3 end (depart) on a particular date, the net change in reservations for that day would be +2. This calculation consolidates all changes into a single value per date, ensuring there's no duplication of dates in the dataset, which is necessary for the next steps.

**2. cumsum()**

- **Purpose:** The cumulative sum (`cumsum()`) takes these daily net changes and accumulates them over the entire period to calculate the total number of active reservations for each date. It essentially adds up the net changes from the start date, rolling forward, to show how many reservations are active on any given day.
- **What It Solves:** This step provides the running total of active reservations. It accounts for the ongoing balance of reservations as they begin and end over time, showing the total active reservations on each date. This is crucial for understanding the capacity or occupancy on any given day.

**Illustrative Example:**

Let's say you have data for three days:

- **Day 1:** 5 arrivals, 0 departures (net +5)
- **Day 2:** 3 arrivals, 1 departure (net +2)
- **Day 3:** 2 arrivals, 4 departures (net -2)

After `groupby().sum()`, you'd have a net change sequence of [+5, +2, -2].

Applying `cumsum()` to this sequence gives you the total active reservations for each day: [5, 7, 5]. This demonstrates how the occupancy evolves:

- **Day 1:** Starts with 5,
- **Day 2:** Increases to 7,
- **Day 3:** Decreases back to 5.

**Conclusion:**

- **`groupby().sum()`** is used for condensing the dataset into a form where each date has a single net change value, resolving any issues with duplicate dates.
- **`cumsum()`** transforms these net changes into a running total of active reservations, reflecting how the number of active reservations builds up or reduces over time.

### Functions: Calculating Occupancies

In [9]:
def get_counts(dataframe, arrivaldate, departuredate, name = 'count'):
    
    '''Generate counts for arrivals and departures on their respective dates.'''
    
    arrivals = (dataframe[arrivaldate]
                .value_counts()
                .rename(name))
    
    departures = (dataframe[departuredate]
                  .value_counts()
                  .rename(name))
    
    return arrivals, departures


def aggregate_counts_by_date(arrivals, departures):
    
    '''Create a DataFrame from arrivals and departures,
    marking departures as negative.'''
    
    df_counts = (pd.concat([arrivals, -departures])
                 .sort_index()
                 .reset_index())
    
    df_counts.columns = ['Date', 'Count']
    
    return df_counts.groupby('Date').sum()


def generate_date_range(df_counts):
    '''Generate a complete date range covering the period in df_data.'''
    
    return pd.date_range(start=df_counts.index.min(),
                         end=df_counts.index.max())


def reindex_and_fill_zero(df_counts, date_range):
    '''Reindex the aggregated count DataFrame to include all dates in the range,
    filling missing dates with 0'''
    
    return df_counts.reindex(date_range, fill_value=0)


def calculate_daily_active_res(df_counts):
    '''Calculate the cumulative sum to determine active reservations for each date'''
   
    return df_counts['Count'].cumsum()


def calculate_daily_occupancy(dataframe, arrivaldate, departuredate, name = 'count'):
    
    arrivals, departures = get_counts(dataframe, arrivaldate, departuredate, name = 'count')
    
    daily_counts = aggregate_counts_by_date(arrivals, departures)
    
    date_range = generate_date_range((daily_counts))
    
    df_reindexed = reindex_and_fill_zero(daily_counts, date_range)
    
    return calculate_daily_active_res(df_reindexed)

# Calculating Occupancy for Hotel 1

In [10]:
df_data.columns

Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'ReservedRoomType', 'AssignedRoomType',
       'BookingChanges', 'DepositType', 'Agent', 'Company',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'ReservationStatus', 'HotelNumber',
       'ArrivalDate', 'DepartureDate', 'Length of Stay', 'BookingDate',
       'ArrivalDate_DaysAfterHoliday', 'ArrivalDate_DaysBeforeHoliday',
       'DepartureDate_DaysAfterHoliday', 'DepartureDate_DaysBeforeHoliday',
       'BookingDate_DaysAfterHoliday', 'BookingDate_DaysBeforeHoliday',
       'ArrivalDate_IsHoliday', 'DepartureDate_IsHoliday',
       'BookingDate_IsHoliday', 'ArrivalDate_Season', 'ArrivalDate_DayOfWeek',
      

In [11]:
## Subset the data

hotel = 'H1'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h1 = df_data[hotel_filter]

df_data_h1

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,ArrivalDate,DepartureDate,Length of Stay,BookingDate,ArrivalDate_DaysAfterHoliday,ArrivalDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,ArrivalDate_IsHoliday,DepartureDate_IsHoliday,BookingDate_IsHoliday,ArrivalDate_Season,ArrivalDate_DayOfWeek,ArrivalDate_IsWeekend,ArrivalDate_DaysUntilWeekend,ArrivalDate_DaysSinceWeekend,ArrivalDate_Quarter,ArrivalDate_Month,ArrivalDate_WeekOfYear,ArrivalDate_DayOfWeek_Sin,ArrivalDate_DayOfWeek_Cos,ArrivalDate_Month_Sin,ArrivalDate_Month_Cos,ArrivalDate_DayOfYear,ArrivalDate_DayOfYear_Sin,ArrivalDate_DayOfYear_Cos,DaysSinceStart,TimeTrend_Normalized,TimeTrend_Squared,TimeTrend_Cubed,Fourier_Annual_Sin_1,Fourier_Annual_Cos_1,Fourier_SemiAnnual_Sin_1,Fourier_SemiAnnual_Cos_1,Fourier_Quarterly_Sin_1,Fourier_Quarterly_Cos_1,Fourier_Monthly_Sin_1,Fourier_Monthly_Cos_1,Fourier_Weekly_Sin_1,Fourier_Weekly_Cos_1,Interaction_Trend_Annual_Sin,Interaction_Trend_Annual_Cos,Interaction_Trend_Month_Sin,Interaction_Trend_Month_Cos,Interaction_LeadTime_Annual_Sin,Interaction_LeadTime_Annual_Cos,Fourier_Annual_Sin_2,Fourier_Annual_Cos_2,Fourier_Annual_Sin_3,Fourier_Annual_Cos_3,Fourier_Monthly_Sin_2,Fourier_Monthly_Cos_2
0,0,342,2015,July,27,1,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,2015-07-01,2015-07-01,0,2014-07-24,21,45,21,45,44,22,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,342,0.00,1.00,0.00,1.00,0.00,1.00
56,0,149,2015,July,27,1,2,1.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,69.60,0,1,Check-Out,H1,2015-07-01,2015-07-08,7,2015-02-02,21,45,28,38,32,60,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,149,0.00,1.00,0.00,1.00,0.00,1.00
57,0,149,2015,July,27,1,2,0.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,55.68,0,0,Check-Out,H1,2015-07-01,2015-07-08,7,2015-02-02,21,45,28,38,32,60,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,149,0.00,1.00,0.00,1.00,0.00,1.00
58,0,145,2015,July,27,1,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,115,NULL,0,Contract,82.35,0,0,Check-Out,H1,2015-07-01,2015-07-08,7,2015-02-06,21,45,28,38,36,56,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,145,0.00,1.00,0.00,1.00,0.00,1.00
59,0,110,2015,July,27,1,3,0.00,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,C,2,No Deposit,6,NULL,0,Transient,79.59,1,1,Check-Out,H1,2015-07-01,2015-07-06,5,2015-03-13,21,45,26,40,71,21,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,110,0.00,1.00,0.00,1.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119384,1,70,2017,August,35,31,2,0.00,0,BB,ESP,Online TA,TA/TO,0

## Calculate Daily and Max Occupancies

In [12]:
df_counts = calculate_daily_occupancy(df_data_h1, 'ArrivalDate', 'DepartureDate')
df_counts.name = 'Active_Reservations'
df_counts.head(10)

2015-07-01     41
2015-07-02     78
2015-07-03    108
2015-07-04    147
2015-07-05    154
2015-07-06    172
2015-07-07    190
2015-07-08    201
2015-07-09    213
2015-07-10    222
Freq: D, Name: Active_Reservations, dtype: int64

In [13]:
# Define active_reservations
active_reservations = df_counts

# Find the maximum occupancy
max_occupancy = active_reservations.max()
max_occupancy

356

## Append Occupancies to Subset Data

In [14]:
# Map the occupancy on arrival and departure dates to each reservation
df_data_h1.loc[:, 'occupancy_at_arrival'] = df_data_h1.loc[:, 'ArrivalDate'].map(active_reservations)
df_data_h1.loc[:, 'occupancy_at_departure'] = df_data_h1.loc[:, 'DepartureDate'].map(active_reservations)

# Convert these occupancies to percentages of the maximum occupancy
df_data_h1.loc[:, 'occupancy_pct_at_arrival'] = (df_data_h1.loc[:, 'occupancy_at_arrival'] / max_occupancy)
df_data_h1.loc[:, 'occupancy_pct_at_departure'] = (df_data_h1.loc[:, 'occupancy_at_departure'] / max_occupancy)

df_data_h1

C:\Users\Ben\AppData\Local\Temp\ipykernel_29364\2601506359.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h1.loc[:, 'occupancy_at_arrival'] = df_data_h1.loc[:, 'ArrivalDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_29364\2601506359.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h1.loc[:, 'occupancy_at_departure'] = df_data_h1.loc[:, 'DepartureDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_29364\2601506359.py:6: SettingWithCopyWa

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,ArrivalDate,DepartureDate,Length of Stay,BookingDate,ArrivalDate_DaysAfterHoliday,ArrivalDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,ArrivalDate_IsHoliday,DepartureDate_IsHoliday,BookingDate_IsHoliday,ArrivalDate_Season,ArrivalDate_DayOfWeek,ArrivalDate_IsWeekend,ArrivalDate_DaysUntilWeekend,ArrivalDate_DaysSinceWeekend,ArrivalDate_Quarter,ArrivalDate_Month,ArrivalDate_WeekOfYear,ArrivalDate_DayOfWeek_Sin,ArrivalDate_DayOfWeek_Cos,ArrivalDate_Month_Sin,ArrivalDate_Month_Cos,ArrivalDate_DayOfYear,ArrivalDate_DayOfYear_Sin,ArrivalDate_DayOfYear_Cos,DaysSinceStart,TimeTrend_Normalized,TimeTrend_Squared,TimeTrend_Cubed,Fourier_Annual_Sin_1,Fourier_Annual_Cos_1,Fourier_SemiAnnual_Sin_1,Fourier_SemiAnnual_Cos_1,Fourier_Quarterly_Sin_1,Fourier_Quarterly_Cos_1,Fourier_Monthly_Sin_1,Fourier_Monthly_Cos_1,Fourier_Weekly_Sin_1,Fourier_Weekly_Cos_1,Interaction_Trend_Annual_Sin,Interaction_Trend_Annual_Cos,Interaction_Trend_Month_Sin,Interaction_Trend_Month_Cos,Interaction_LeadTime_Annual_Sin,Interaction_LeadTime_Annual_Cos,Fourier_Annual_Sin_2,Fourier_Annual_Cos_2,Fourier_Annual_Sin_3,Fourier_Annual_Cos_3,Fourier_Monthly_Sin_2,Fourier_Monthly_Cos_2,occupancy_at_arrival,occupancy_at_departure,occupancy_pct_at_arrival,occupancy_pct_at_departure
0,0,342,2015,July,27,1,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,2015-07-01,2015-07-01,0,2014-07-24,21,45,21,45,44,22,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,342,0.00,1.00,0.00,1.00,0.00,1.00,41,41,0.12,0.12
56,0,149,2015,July,27,1,2,1.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,69.60,0,1,Check-Out,H1,2015-07-01,2015-07-08,7,2015-02-02,21,45,28,38,32,60,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,149,0.00,1.00,0.00,1.00,0.00,1.00,41,201,0.12,0.56
57,0,149,2015,July,27,1,2,0.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,55.68,0,0,Check-Out,H1,2015-07-01,2015-07-08,7,2015-02-02,21,45,28,38,32,60,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,149,0.00,1.00,0.00,1.00,0.00,1.00,41,201,0.12,0.56
58,0,145,2015,July,27,1,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,115,NULL,0,Contract,82.35,0,0,Check-Out,H1,2015-07-01,2015-07-08,7,2015-02-06,21,45,28,38,36,56,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,145,0.00,1.00,0.00,1.00,0.00,1.00,41,201,0.12,0.56
59,0,110,2015,July,27,1,3,0.00,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,C,2,No Deposit,6,NULL,0,Transient,79.59,1,1,Check-Out,H1,2015-07-01,2015-07-06,5,2015-03-13,21,45,26,40,71,21,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,110,0.00,1.00,0.00,1.00,0.00,1.00,41,172,0.12,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# Calculating Occupancy for Hotel 2

In [15]:
hotel = 'H2'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h2 = df_data[hotel_filter]

df_counts_h2 = calculate_daily_occupancy(df_data_h2, 'ArrivalDate', 'DepartureDate')
df_counts_h2.name = 'Active_Reservations'
df_counts_h2

2015-07-01     79
2015-07-02    128
2015-07-03     71
2015-07-04     68
2015-07-05     57
             ... 
2017-09-07      2
2017-09-08      2
2017-09-09      1
2017-09-10      1
2017-09-11      0
Freq: D, Name: Active_Reservations, Length: 804, dtype: int64

## Calculate Daily and Max Occupancies

In [16]:
# Define active_reservations
active_reservations = df_counts

# Find the maximum occupancy
max_occupancy = active_reservations.max()

## Append Occupancies to Subset Data

In [17]:
# Map the occupancy on arrival and departure dates to each reservation
df_data_h2.loc[:, 'occupancy_at_arrival'] = df_data_h2.loc[:, 'ArrivalDate'].map(active_reservations)
df_data_h2.loc[:, 'occupancy_at_departure'] = df_data_h2.loc[:, 'DepartureDate'].map(active_reservations)

# Convert these occupancies to percentages of the maximum occupancy
df_data_h2.loc[:, 'occupancy_pct_at_arrival'] = (df_data_h2.loc[:, 'occupancy_at_arrival'] / max_occupancy)
df_data_h2.loc[:, 'occupancy_pct_at_departure'] = (df_data_h2.loc[:, 'occupancy_at_departure'] / max_occupancy)

df_data_h2

C:\Users\Ben\AppData\Local\Temp\ipykernel_29364\3546143305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h2.loc[:, 'occupancy_at_arrival'] = df_data_h2.loc[:, 'ArrivalDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_29364\3546143305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h2.loc[:, 'occupancy_at_departure'] = df_data_h2.loc[:, 'DepartureDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_29364\3546143305.py:6: SettingWithCopyWa

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,ArrivalDate,DepartureDate,Length of Stay,BookingDate,ArrivalDate_DaysAfterHoliday,ArrivalDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,ArrivalDate_IsHoliday,DepartureDate_IsHoliday,BookingDate_IsHoliday,ArrivalDate_Season,ArrivalDate_DayOfWeek,ArrivalDate_IsWeekend,ArrivalDate_DaysUntilWeekend,ArrivalDate_DaysSinceWeekend,ArrivalDate_Quarter,ArrivalDate_Month,ArrivalDate_WeekOfYear,ArrivalDate_DayOfWeek_Sin,ArrivalDate_DayOfWeek_Cos,ArrivalDate_Month_Sin,ArrivalDate_Month_Cos,ArrivalDate_DayOfYear,ArrivalDate_DayOfYear_Sin,ArrivalDate_DayOfYear_Cos,DaysSinceStart,TimeTrend_Normalized,TimeTrend_Squared,TimeTrend_Cubed,Fourier_Annual_Sin_1,Fourier_Annual_Cos_1,Fourier_SemiAnnual_Sin_1,Fourier_SemiAnnual_Cos_1,Fourier_Quarterly_Sin_1,Fourier_Quarterly_Cos_1,Fourier_Monthly_Sin_1,Fourier_Monthly_Cos_1,Fourier_Weekly_Sin_1,Fourier_Weekly_Cos_1,Interaction_Trend_Annual_Sin,Interaction_Trend_Annual_Cos,Interaction_Trend_Month_Sin,Interaction_Trend_Month_Cos,Interaction_LeadTime_Annual_Sin,Interaction_LeadTime_Annual_Cos,Fourier_Annual_Sin_2,Fourier_Annual_Cos_2,Fourier_Annual_Sin_3,Fourier_Annual_Cos_3,Fourier_Monthly_Sin_2,Fourier_Monthly_Cos_2,occupancy_at_arrival,occupancy_at_departure,occupancy_pct_at_arrival,occupancy_pct_at_departure
1,0,257,2015,July,27,1,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6,NULL,0,Transient,80.00,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00,41,108,0.12,0.30
2,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00,41,108,0.12,0.30
3,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00,41,108,0.12,0.30
4,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00,41,108,0.12,0.30
5,0,257,2015,July,27,1,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,2015-07-01,2015-07-03,2,2014-10-17,21,45,23,43,63,52,False,False,False,Summer,3,False,2,2,3,7,27,0.43,-0.90,-0.50,-0.87,182,0.01,-1.00,0,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,-0.00,-0.00,0,257,0.00,1.00,0.00,1.00,0.00,1.00,41,108,0.12,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Combine Subset Datasets into Single DataFrame

In [18]:
full_dataset = pd.concat([df_data_h1, df_data_h2], axis = 0)

full_dataset.to_parquet('../../data/3.2_data_with_occupancies.parquet', compression = 'zstd')

# **Final Review**

---

The calculation of occupancy percentages at the time of arrival and departure has provided valuable insights into the hotel’s operational dynamics during guests' stays. By determining occupancy levels at these critical points, I've gained a clearer understanding of how room availability fluctuates and how it might influence guest behavior, including cancellations. 

These occupancy metrics will play an important role in refining the predictive models and improving the accuracy of forecasts related to booking patterns, cancellations, and revenue management. The addition of these features strengthens the dataset, making it more robust for the next stages of analysis and modeling.

---